## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-19-46-50 +0000,bbc,Rapidly growing fire near Grand Canyon prompts...,https://www.bbc.com/news/articles/czjk10wwmkeo
1,2025-07-12-19-40-22 +0000,nyt,Rubio Visits Asia in Shadow of Trump’s Tariffs,https://www.nytimes.com/video/world/asia/10000...
2,2025-07-12-19-38-25 +0000,bbc,"Arizona resident dies from plague, health offi...",https://www.bbc.com/news/articles/cz7l8w8881qo
3,2025-07-12-18-31-14 +0000,bbc,Trump says EU and Mexico face 30% tariff from ...,https://www.bbc.com/news/articles/cyvj13d9ylpo
4,2025-07-12-17-23-07 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2306/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
12,trump,8
24,air,6
25,india,6
28,report,3
27,crash,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
5,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...,28
4,2025-07-12-17-23-07 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...,27
12,2025-07-12-10-47-08 +0000,bbc,'Was this avoidable?': Families of Air India c...,https://www.bbc.com/news/articles/c5ylv04r1eyo,24
6,2025-07-12-16-50-32 +0000,bbc,Mystery of Air India crash deepens as report r...,https://www.bbc.com/news/articles/cx2gy78gpnqo,23
16,2025-07-12-06-04-07 +0000,bbc,"Air India report may lead to answers, but not ...",https://www.bbc.com/news/videos/c80pj0pv974o,22


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
5,28,2025-07-12-17-01-00 +0000,wsj,The U.S. will charge a 30% tariff on the Europ...,https://www.wsj.com/economy/trade/trump-threat...
4,27,2025-07-12-17-23-07 +0000,nyt,Air India Plane Crash Report Says Fuel to Engi...,https://www.nytimes.com/2025/07/11/world/asia/...
22,15,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
9,13,2025-07-12-14-00-00 +0000,wsj,The White House has an explanation for the nea...,https://www.wsj.com/politics/policy/trump-new-...
15,12,2025-07-12-09-30-00 +0000,wsj,Pittsburgh is striving to turn its old steel m...,https://www.wsj.com/tech/ai/can-pittsburghs-ol...
23,12,2025-07-12-01-12-57 +0000,bbc,'They were just kids': Mother mourns sons kill...,https://www.bbc.com/news/articles/cy9xgrrq54go
17,10,2025-07-12-05-19-04 +0000,bbc,Al Fayed family battle over luxury penthouse w...,https://www.bbc.com/news/articles/c0m8pew4331o
28,10,2025-07-12-00-33-38 +0000,bbc,Med Sea heatwave might feel nice for holiday s...,https://www.bbc.com/news/articles/c4g2v1l7j6yo
33,10,2025-07-11-23-36-00 +0000,bbc,Flight from London to Cancun lands in US after...,https://www.bbc.com/news/articles/c98jqve2q7mo
42,9,2025-07-11-20-40-22 +0000,nyt,Son of El Chapo Pleads Guilty to Sweeping Drug...,https://www.nytimes.com/2025/07/11/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
